In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def load_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    return df, iris

##Create widgets

In [ ]:
def create_widgets(iris):
    # Tab 1: Input Parameters
    knn_slider = widgets.FloatSlider(value=3, min=1, max=10, step=1, description='K (Neighbors):')
    test_size_slider = widgets.FloatSlider(value=0.2, min=0.1, max=0.5, step=0.05, description='Test Size:')

    # Tab 2: Data Filter
    species_dropdown = widgets.Dropdown(options=iris.target_names.tolist(), description='Species Filter:')
    sample_size_text = widgets.Text(value='10', description='Sample Size:')

    # Tab 3: Action Button
    run_button = widgets.Button(description='Run Model', button_style='success')

    show_plot_checkbox = widgets.Checkbox(value=True, description='Show Feature Plot')


    return knn_slider, test_size_slider, species_dropdown, sample_size_text, run_button, show_plot_checkbox


##Model Execution

In [ ]:
def run_model(df, iris, k, test_size, species_filter, sample_size):
    # Filter
    species_idx = list(iris.target_names).index(species_filter)
    df_filtered = df[df['target'] == species_idx]

    # Sample
    sample_size = min(int(sample_size), len(df_filtered))
    df_sample = df_filtered.sample(sample_size, random_state=42)

    # Split
    X = df_sample.iloc[:, :-1]
    y = df_sample['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train & Predict
    model = KNeighborsClassifier(n_neighbors=int(k))
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    return acc, sample_size, len(X_train), len(X_test)

##UI Display

In [ ]:
def display_ui():
    df, iris = load_data()
    knn_slider, test_size_slider, species_dropdown, sample_size_text, run_button, show_plot_checkbox = create_widgets(iris)
    output_area = widgets.Output()

    def on_run_clicked(b):
        with output_area:
            clear_output()
            try:
                acc, total, trai                                            n_n, test_n = run_model(
                    df, iris,
                    k=knn_slider.value,
                    test_size=test_size_slider.value,
                    species_filter=species_dropdown.value,
                    sample_size=sample_size_text.value
                )
                print(f" Model trained on {total} samples")
                print(f"Train size: {train_n}, Test size: {test_n}")
                print(f" Accuracy: {acc * 100:.2f}%")

                if show_plot_checkbox.value:
                    import matplotlib.pyplot as plt
                    import seaborn as sns
                    species_idx = list(iris.target_names).index(species_dropdown.value)
                    df_filtered = df[df['target'] == species_idx]

                    plt.figure(figsize=(6, 4))
                    sns.scatterplot(
                        data=df_filtered,
                        x=iris.feature_names[0],
                        y=iris.feature_names[2],
                        hue=df_filtered['target'],
                        palette='Set1'
                    )
                    plt.title(f'{species_dropdown.value} Distribution')
                    plt.show()
            except Exception as e:
                print("Error:", e)

    run_button.on_click(on_run_clicked)

    # Organize Tabs
    tab1 = widgets.VBox([knn_slider, test_size_slider])
    tab2 = widgets.VBox([species_dropdown, sample_size_text, show_plot_checkbox])
    tab3 = widgets.VBox([run_button, output_area])

    tabs = widgets.Tab(children=[tab1, tab2, tab3])
    tabs.set_title(0, 'Model Params')
    tabs.set_title(1, 'Data Filter')
    tabs.set_title(2, 'Run & Output')

    display(tabs)

In [ ]:
display_ui()